# Accessible Green & Blue Infrastructure across England

Below are the data sources for features relating to Green and Blue infrastructure in England

 NE Green Infrastructure layers (open) Accessible Greenspace

https://www.data.gov.uk/dataset/f335ab3a-f670-467f-bedd-80bdd8f1ace6/green-and-blue-infrastructure-england

https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/Natural_England/Access_Green_Infrastructure/Green_and_Blue_Infrastructure_NE/Green_and_Blue_Infrastructure_Opendata_NE_Geopackage.zip

Download provides accompanying metadat for dataset where details of the features of each layer in the geopkg can be found:
Green_and_Blue_Infrastructure_Opendata_NE_Geopackage/GI_Access_Maps/Metadata/Green_Infrastructure_Access_Maps_Open_Data_Attribute_Metadata.pdf

In [ ]:
%load_ext autoreload
%autoreload 2

# All the variables are defined in the Config file
from model_config import *
from model_packages import *
from model_utils import *

In [ ]:
os.chdir('..')

# Feature Extraction

## Green Infrastructure

A series of geodatabases that contain a range of spatial datasets. These datasets describe the location and geographical extent of different types of Green and Blue Infrastructure across England. The datasets highlight accessibility levels, display greenspace provision and natural greenspace standards in a spatial context and present it alongside a wide range of social statistics. 

In [ ]:
# We have to choose appropriate layers

for layername in fiona.listlayers(GI_Access_maps):
    with fiona.open(GI_Access_maps, layer=layername) as src:
        print(layername, len(src))

In [ ]:
# Spatial dataset describing the location and geographical extent of different types
# of Green and Blue Infrastructure across England, based on a systematic typology system.
# This includes both accessible and non-accessible Green Infrastructure areas.

geopkg = gpd.read_file(GI_Access_maps,layer='Map1_OGL')

accessible_geopkg=geopkg[(geopkg['Accessible']=='Yes')].reset_index(drop=True)

accessible_geopkg=accessible_geopkg[['TypologyTitle','geometry']]

sites_df=gpd.read_file(data_folder+'accessibility.shp')

sites_df=sites_df[sites_df['geom_type']=='5km buffer'].reset_index(drop=True)

accessible_geopkg_sites = accessible_geopkg.to_crs(crs_mtr).\
overlay(sites_df[[x for x in sites_df.columns if x not in ['area']]].to_crs(crs_mtr), how='intersection')

# save dataframe of accessible green and blue infrastructure sites intersected with 5km buffer zones around people counter locations
accessible_geopkg_sites.to_pickle(data_folder+'accessible_geopkg_sites.pkl')

In [ ]:
# visualisation of accessible green spaces in the vicinity of the people counter sites
world = gpd.read_file(world_boundaries)

uk = world[world.name == 'U.K. of Great Britain and Northern Ireland'] 

uk=uk.to_crs(crs_deg)


ax=uk.plot(alpha=0.1,figsize=(5,10))
accessible_geopkg_sites.to_crs(crs_deg).plot(ax=ax,alpha=1,color='green')

## Accessible woodlands

In [ ]:
# accessible woodlands

# This spatial dataset shows the geographical extent and location of woodlands across England.
# The dataset also highlights accessible woodlands along with showing non-accessible woodlands
# and woodlands with linear access by ProW.

geopkg_4 = gpd.read_file(GI_Access_maps,layer='Map4')
geopkg_4=geopkg_4[geopkg_4['AccessLevel']!='Non Accessible']


accessible_geopkg_4_sites = geopkg_4.to_crs(crs_mtr).\
overlay(sites_df[[x for x in sites_df.columns if x not in ['area']]].to_crs(crs_mtr), how='intersection')

In [ ]:
world = gpd.read_file(world_boundaries)

uk = world[world.name == 'U.K. of Great Britain and Northern Ireland'] 

uk=uk.to_crs(crs_deg)


ax=uk.plot(alpha=0.1,figsize=(5,5))
accessible_geopkg_4_sites.to_crs(crs_deg).plot(ax=ax,alpha=1,color='green')

## Public right of Way

In [ ]:
# ProW Density Grid (PRoW Density (m per 1km square))
# This spatial dataset is a 1km grid of England showing 
# the density of ProW each grid square.
geopkg_5 = gpd.read_file(GI_Access_maps,layer='England_ProwDensity_1kmGrid')

geopkg_5=geopkg_5[['PROW_Total_length_m','geometry']]

accessible_geopkg_5_sites = geopkg_5.to_crs(crs_mtr).\
overlay(sites_df[[x for x in sites_df.columns if x not in ['area']]].to_crs(crs_mtr), how='intersection')

accessible_geopkg_5_sites.to_pickle(data_folder+'accessible_geopkg_5_sites.pkl')

# create total lenght of PROW for each counter buffer zone in Km
accessible_geopkg_5_sites=accessible_geopkg_5_sites.groupby('counter')['PROW_Total_length_m'].sum().reset_index()
accessible_geopkg_5_sites['PROW_Total_length_km']=accessible_geopkg_5_sites['PROW_Total_length_m']/(10**3)

## Inland Waterside

In [ ]:
#Inland waterside – ProW ANG
#This spatial dataset shows the geographical extent
#and location of inland waterside across England that
#intersects with the ProW network and areas of Accessible Natural Greenspace


for layername in fiona.listlayers(Blue_Infrastructure_Waterside):
    with fiona.open(Blue_Infrastructure_Waterside, layer=layername) as src:
        print(layername, len(src))

In [ ]:
# read layer corresponding to Inland Waterside PROW
geopkg_wtr = gpd.read_file(Blue_Infrastructure_Waterside,layer='Inland_Waterside_PROW_ANG')

# find the intersection of inland waterside PROW with people counter buffer zones
geopkg_wtr_sites = geopkg_wtr.to_crs(crs_mtr).\
overlay(sites_df[[x for x in sites_df.columns if x not in ['area']]].to_crs(crs_mtr), how='intersection')

#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.length.html

geopkg_wtr_sites=geopkg_wtr_sites[['counter','geometry']]

# find the length of each instance inland waterside PROW within buffer zone 
geopkg_wtr_sites['length']=geopkg_wtr_sites.length

geopkg_wtr_sites.to_pickle(data_folder+'geopkg_wtr_sites.pkl')

# create total lenght of inland waterside access for each counter buffer zone in Km
geopkg_wtr_sites=geopkg_wtr_sites.groupby('counter')['length'].sum().reset_index()
geopkg_wtr_sites['waterside_length_km']=geopkg_wtr_sites['length']/(10**3)

## Collating all features: Accessible Greenspaces, Public rights of way, accessible inland waterside

In [ ]:
accessible_geopkg_sites=pd.read_pickle(data_folder+'accessible_geopkg_sites.pkl')
accessible_geopkg_sites['accessible_green_space_area']=accessible_geopkg_sites.area
accessible_geopkg_sites=accessible_geopkg_sites.groupby('counter')['accessible_green_space_area'].\
sum().reset_index()
accessible_geopkg_sites['accessible_green_space_area']=accessible_geopkg_sites['accessible_green_space_area']/\
(10**6)

accessible_geopkg_5_sites=pd.read_pickle(data_folder+'accessible_geopkg_5_sites.pkl')
accessible_geopkg_5_sites=accessible_geopkg_5_sites.groupby('counter')['PROW_Total_length_m'].sum().reset_index()
accessible_geopkg_5_sites['PROW_Total_length_km']=accessible_geopkg_5_sites['PROW_Total_length_m']/(10**3)

geopkg_wtr_sites=pd.read_pickle(data_folder+'geopkg_wtr_sites.pkl')
geopkg_wtr_sites=geopkg_wtr_sites.groupby('counter')['length'].sum().reset_index()
geopkg_wtr_sites['waterside_length_km']=geopkg_wtr_sites['length']/(10**3)

world = gpd.read_file(world_boundaries)

uk = world[world.name == 'U.K. of Great Britain and Northern Ireland'] 

uk=uk.to_crs(crs_deg)



## Save and export all features 

In [ ]:
# join all features

green_blue_prows_ftrs=[accessible_geopkg_sites,geopkg_wtr_sites,accessible_geopkg_5_sites]


df_green_blue_prows_ftrs= reduce(lambda  left,right: pd.merge(left,right,on=['counter'],how='inner'),\
                                 green_blue_prows_ftrs)

In [ ]:
df_green_blue_prows_ftrs.to_pickle(data_folder+'green_blue_prow_ftrs.pkl')

In [ ]:
df_green_blue_prows_ftrs.shape